# Init

In [2]:
# imports
import gurobipy as gp
from gurobipy import GRB
from collections import defaultdict

---
#### Load data

In [3]:
num_video = 0
num_endpoint = 0
num_req = 0
num_server = 0

cache_capacity = 0
video_size = []

latency = defaultdict(dict)                         # [endpoint][cache/datacenter]
reqs = defaultdict(lambda: defaultdict(int))        # [endpoint][video][num reqs]

In [4]:
# dataset = "dataset/videos_worth_spreading.in"
dataset = "dataset/me_at_the_zoo.in"

status = 0
endpoint_index = 0
num_connected_cache = 0
with open(dataset, "r") as f:
    for line_content in f:
        line = line_content.split()

        if status ==0:                                  # get counters
            num_video = int(line[0])
            num_endpoint = int(line[1])
            num_req = int(line[2])
            num_cache = int(line[3])
            cache_capacity = int(line[4])
            status = 1

        elif status == 1:                               # get video dims
            for size in line:
                video_size.append(int(size))
            status = 2

        elif status == 2:                               # get datacenter latency and connected cache number
            data_center_latency = int(line[0])
            latency[endpoint_index][num_cache + 1] = data_center_latency
            
            num_connected_cache = int(line[1])
            if not num_connected_cache:
                endpoint_index = endpoint_index + 1
                if endpoint_index == num_endpoint:
                    status = 4
            else:
                status = 3
        
        elif status == 3:                                  # get cache latency
            cache_index = int(line[0])
            cache_latency = int(line[1])
            latency[endpoint_index][cache_index] = cache_latency
            
            num_connected_cache = num_connected_cache - 1
            if not num_connected_cache:
                endpoint_index = endpoint_index + 1
                if endpoint_index == num_endpoint:
                    status = 4
                else:
                    status = 2
        
        elif status == 4:
            video_index = int(line[0])
            endpoint_index = int(line[1])
            num_reqs = int(line[2])
            reqs[endpoint_index][video_index] = num_reqs               # [endpoint][video][num reqs]            

In [5]:
print(reqs[4][27])

340


---
# Math model

In [6]:
endpoint_index = range(num_endpoint)
server_index = range(num_server + 1) # I've modelled datacenter as last server
video_index = range(num_video)

In [10]:
model = gp.Model("YoutubeCache")

# decision vars
x = model.addVars(endpoint_index, server_index, video_index, vtype=gp.GRB.BINARY, name="x")
y = model.addVars(server_index, video_index, vtype=gp.GRB.BINARY, name="y")

# objective function
obj = gp.quicksum(latency[e][s]*reqs[e][v]*x[e,s,v] for e in endpoint_index for s in server_index for v in video_index)
model.setObjective(obj, GRB.MINIMIZE)

# constraints
constr = gp.quicksum( (x[e,s,v] for e in endpoint_index) <= num_endpoint*y[s,v] for s in server_index for v in video_index ) 
model.addConstr(constr, name="if video v available on server s constraint")

constr = gp.quicksum() # TODO: check how to do this in a more efficient way (or use a new variable if not possible)
model.addConstr(constr, name="CONSTRAINT EVERY REQUEST SATISFIED")

# TODO: check if is better to set datacenter capacity as inf instead of -1 approach
constr = gp.quicksum( (video_size[v] * y[s,v] for v in video_index) <= cache_capacity for s in (server_index - 1)) # -1 because datacenter have all the video
model.addConstr(constr, name="cache capacity constraint")


constr = gp.quicksum( y[num_cache,v] == 1 for v in video_index) # remember that cache servers are from 0 to n-1, n index is for datacenter
model.addConstr(constr, name="Datacenter have all videos constraint")

constr = gp.quicksum( (x[e,s,v] for v in video_index) <= num_video*latency[e,s] for e in endpoint_index for s in server_index)
model.addConstr(constr, name="if video v available on server s constraint")




KeyError: 0